In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
%matplotlib inline

In [3]:
def vocab2vec(vocab_size, vocab_length=10**7):
    f = open("Combined_String.txt", "r")
    s = f.read()
    f.close()
    D = 'abcdefghijklmnopqrstuvwxyz .,\'1234567890";'
    res = []
    for i in range(vocab_length):
        c = s[i].lower()
        v = np.zeros((vocab_size))
        try:
            idx = D.index(c)
            v[idx] = 1
            res.append(v)
        except (ValueError, IndexError) as e:
            pass
        
        
    ret = np.array(res) # A list of shape (vocab_length,) one-hot encoded characters
    print ("shape is: {}".format(ret.shape))
    return ret

#vocab2vec(40)

# What is LSTM? #  
- [Nico's blog on LSTM](http://nicodjimenez.github.io/2014/08/08/lstm.html)  
- [Colah's blog on LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs)  
- Written in equation:
    Each LSTM cell has three inputs (character $x_t$, cell prediction $h_{t-1}$, and hidden state $C_{t-1}$) and two outputs (hidden state $C_t$ and cell prediction $h_t$).  
    Forget gate: 
    $$f_t = \sigma (W_f [h_{t-1}, x_t] + b_f)$$   
    Information gate: 
    $$i_t = \sigma (W_i [h_{t-1}, x_t] + b_i)$$  
    Updates for cell state:
    $$D_t = tanh (W_D [h_{t-1}, x_t] + b_D)$$  
    $$C_t = f_t * C_{t-1} + i_t * D_t$$
    Output layers:  
    $$o_t = \sigma (W_o [h_{t-1}, x_t] + b_o)$$
    $$h_t = o_t * tanh(C_t)$$
    
- Training goal:  
    $argmin_W J$, where
    $$J = \sum_t (y_t log h_t)$$

**LSTM used for reading paragraphs character by character**
- Cell prediction is the next (batch of) character, given the input and previous states.
- The first character in prediction sequence is used to calculate the cross entropy.


**Existing code examples:**  
- [Aymeric Damien's TensorFlow-Examples](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py)  

In [25]:
# My implementation of LSTM on character reading - based on the equation above

class BasicLSTM:
    def __init__(self, vocab_size, cell_size, batch_size, continue_training = False, global_step = -1):
        self.batch_size = batch_size
        self.vocab_size = vocab_size
        self.cell_size = cell_size
        self.global_step = global_step
        
        self.global_step = global_step
        self.MODEL_NAME = "./model/LSTM"
        self.TEST_SAMPLE_SEQ_LENGTH = 100
        self._construct_networks(vocab_size, cell_size, batch_size, continue_training, global_step)
        
        
        
    def _weight(self, shape, dtype=tf.float32, name=None):
        m = 0
        s = 0.01
        return tf.Variable(tf.random_normal(shape=shape, mean=m, stddev=s, dtype=dtype), dtype, name=name)
    
    def _const(self, shape, name, dtype=tf.float32):
        d0 = shape[0]
        d1 = shape[1]
        tmp = np.zeros(shape=shape)
        tmp[:, 0] = np.ones(shape=[d0, 1])
        return tf.constant(tmp, dtype=dtype, name=name)
    
    def _ohe2char(self, ohe_vec): # takes only the first row in ohe_vec
        assert ohe_vec.shape[1] == self.vocab_size
        chars = 'abcdefghijklmnopqrstuvwxyz .,\'1234567890";'
        choice_id = np.random.choice(self.vocab_size, p=ohe_vec[0,:].ravel())
        return chars[choice_id]
            

    def _construct_networks(self, vocab_size, cell_size, batch_size, continue_training, global_step):
        graph = tf.Graph()
        with graph.as_default():
            x = tf.placeholder(tf.float32, [batch_size, vocab_size], name="x")
            y = tf.placeholder(tf.float32, [batch_size, vocab_size], name="y")
            init_C = tf.placeholder(tf.float32, [batch_size, cell_size], name="init_C")
            init_h = tf.placeholder(tf.float32, [batch_size, vocab_size], name="init_h")
        
            if not continue_training:
                # Fotget gate
                Wf = self._weight([2 * vocab_size, cell_size], name="Wf")
                bf = self._const([1, cell_size], name="bf")
                f = tf.nn.softmax(tf.matmul(tf.concat([init_h, x], axis=1), Wf) + bf, dim=1)

                # Info gate
                Wi = self._weight([2 * vocab_size, cell_size], name="Wi")
                bi = self._const([1, cell_size], name="bi")
                i = tf.nn.softmax(tf.matmul(tf.concat([init_h, x], axis=1), Wi) + bi, dim=1)

                # Next cell state
                Wd = self._weight([2 * vocab_size, cell_size], name="Wd")
                bd = self._const([1, cell_size], name="bd")
                D = tf.tanh(tf.matmul(tf.concat([init_h, x], axis=1), Wd) + bd)

                # Update cell state
                C = tf.add(f * init_C, i * D, name="C")

                # Output layers
                Wo = self._weight([2 * vocab_size, vocab_size], name="Wo")
                bo = self._const([1, vocab_size], name="bo")
                o = tf.nn.softmax(tf.matmul(tf.concat([init_h, x], axis=1), Wo) + bo, dim=1)
                h = tf.multiply(o, tf.tanh(C), name="h")
                hs = tf.nn.softmax(h)

                # Loss function, etc.
                #hs = tf.nn.softmax(h, dim=1) # Convert h into softmax form
                loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=h, name="loss"))
                optimizer = tf.train.AdamOptimizer()
                grad_limit = tf.constant(5.0, dtype=tf.float32, name="grad_limit")
                grads_and_vars = optimizer.compute_gradients(loss)
                clipped_grads_and_vars = []
                for grad, var in grads_and_vars:

                    clipped_grad = tf.clip_by_value(grad, -grad_limit, grad_limit)
                    clipped_grads_and_vars.append((clipped_grad, var))
                train_step = optimizer.apply_gradients(clipped_grads_and_vars, name="train")


                # Session, Saver, etc.
                saver = tf.train.Saver()
                sess = tf.Session()
                sess.run(tf.global_variables_initializer())
                saver.save(sess, self.MODEL_NAME,global_step=0)

            else:
                sess = tf.Session()

                saver = tf.train.import_meta_graph(self.MODEL_NAME + "-{}.meta".format(global_step))
                saver.restore(sess,tf.train.latest_checkpoint('./'))

                graph = tf.get_default_graph()

                x = graph.get_tensor_by_name("x:0")
                y = graph.get_tensor_by_name("y:0")
                init_C = graph.get_tensor_by_name("init_C:0")
                init_h = graph.get_tensor_by_name("init_h:0")
                C = graph.get_tensor_by_name("C:0")
                h = graph.get_tensor_by_name("h:0")
                Wf = graph.get_tensor_by_name("Wf:0")
                Wi = graph.get_tensor_by_name("Wi:0")
                Wd = graph.get_tensor_by_name("Wd:0")
                Wo = graph.get_tensor_by_name("Wo:0")
                J = graph.get_tensor_by_name("J:0")

                train_step = graph.get_tensor_by_name("train:0")
            
        
        # After creation, save to class variables
        self.x = x
        self.y = y
        self.init_C = init_C
        self.init_h = init_h
        self.C = C
        self.h = h
        self.hs = hs
        self.loss = loss
        self.train_step = train_step
        self.saver = saver
        self.sess = sess
        
        

        
    def train(self, steps, training_data, sample = True, sample_every = 200000):
        save_per_steps = 10
        batch_size = self.batch_size
        vocab_size = self.vocab_size
        cell_size = self.cell_size
        
        for stp in range(steps):
            
            prev_C = np.random.rand(batch_size, cell_size)
            prev_h = np.random.rand(batch_size, vocab_size)
            p = 0
            while p < (len(training_data) - batch_size - 1):
                self.global_step
                fdata = {self.init_C: prev_C, 
                         self.init_h: prev_h,
                         self.x: training_data[p : p + batch_size], 
                         self.y: training_data[p+1 : p+1+batch_size]
                         }
                _, prev_C, prev_h, loss = self.sess.run([self.train_step, self.C, self.h, self.loss], feed_dict = fdata)
                
                p += batch_size
                
            
                if sample and p % sample_every == 0:
                    # Perform a trial of sample run 
                    words_outputs = ""
                    for i in range(self.TEST_SAMPLE_SEQ_LENGTH):
                        fdata = {self.init_C: prev_C,
                                self.init_h: prev_h,
                                self.x: training_data[p : p + batch_size],
                                 self.y: training_data[p+1 : p+1+batch_size]
                                }
                        _, vec_ohe, loss = self.sess.run([self.C, self.hs, self.loss], feed_dict = fdata)

                        words_outputs += self._ohe2char(vec_ohe)
                    
                    print ("--- n = {}, p = {}, loss = {} ---".format(self.global_step, p, loss))
                    print ("{}\n".format(words_outputs))
                    
                    
            if self.global_step % save_per_steps == 0:
                self.saver.save(self.sess, self.MODEL_NAME, global_step = self.global_step)
        
            self.global_step += 1
    
    

In [24]:
if __name__ == "__main__":
    print ("Started!")
    lstm = BasicLSTM(vocab_size = 40,
                    cell_size = 40, # They have to be equal. GGWP
                    batch_size = 10,
                    continue_training = False,
                    global_step = -1)
    training_words = vocab2vec(40, 10 ** 6)
    lstm.train(steps = 10**5, training_data = training_words, sample = True)
    
    

Started!
shape is: (992170, 40)
--- n = -1, p = 200000, loss = 3.5475053787231445 ---
w ll,a3c6' 30sh,9qs1j5jbzmvo, tpooau719bj ,c17xbal27 r3itisd8lj'hkveodxf8x9mz3c5qr5.94tfl.7nn0hponex

--- n = -1, p = 400000, loss = 3.530280590057373 ---
a 51a'a9th7a098fgxbw,i,ksi0da42vilwlhnkj.9y1gzxez4kvi'38t,uqt.o.e'0yaf.a0qubm8l'tkmf,tk3b837tvut62oo

--- n = -1, p = 600000, loss = 3.3746063709259033 ---
t ,2ffigdh guyd4zii2a6v3wh842.cy gi9edx 3e1,l3.xbpnuhviu j,3vvo q.ige'7yz60cgc8gy9h3mf .n.2 at8x54zk

--- n = -1, p = 800000, loss = 3.722846031188965 ---
f xrmjjgcbcebz2wu9a,msdd'1'aytxslruixjr48.lggeji9i lll9s kmt1lbj2hq82uu 6. r pymv6aze37jo'xit28  1kx

--- n = 0, p = 200000, loss = 3.5466442108154297 ---
2skqa.vu'fu42qsdz79db2ngvimavrv'w624l9twnkvthktgqehy,ke7xoe4 utsap11uouz3,d lx21' q931'n13.ysew5z8na

--- n = 0, p = 400000, loss = 3.5309743881225586 ---
02byph.0b9qcprf7z31ad4uw5s36i.'a1cao3fj3naj4ne qkpyge.7,jnf5nu.pwtq6eay1s683l6u5pwcbspa'ma,a91p3zbnj

--- n = 0, p = 600000, loss = 3.3932

KeyboardInterrupt: 

In [ ]:
if __name__ == "__main__":
    print ("Started!")
    lstm = BasicLSTM(vocab_size = 40,
                    cell_size = 40, # They have to be equal. GGWP
                    batch_size = 1,
                    continue_training = False,
                    global_step = -1)
    training_words = vocab2vec(40, 10 ** 7)
    lstm.train(steps = 10**5, training_data = training_words, sample = True, sample_every = 200000)

Started!
shape is: (9919422, 40)
--- n = -1, p = 200000, loss = 2.95654296875 ---
g6cj 7bjyxwcgv8'skgr1h3l 6 29x7c2okfq'.ougq911qe2ank5tl'nrivbx5h1p5s5yyt bb2v'8cl,jewnwsadh9  r2exbx

--- n = -1, p = 400000, loss = 3.7170217037200928 ---
dcldp za7pwjm2'5gpd8ezej1ai'lohyl3x65pema7wk,.rk.5jstjumcs9p60qcgk3e,q, t5agg.i'wvo9ksb2 u.ajkhar'03

--- n = -1, p = 600000, loss = 2.955446720123291 ---
n1sex xdbq1't4w2yn85e,3l9a88pfkt20zfkdvf.pm1qtyu4sr2wvh69bxokibz.5dwjnlf,3yfbbi5pbx168l34k2xi5dqpc ,

--- n = -1, p = 800000, loss = 3.6754627227783203 ---
f7jxxixyyyvjtb5quwbmh2.g9widj3uublx0dm11aq'00oztb9jccd wum0 o p9a1y9jfosdg4au5z3whja,vq5o 7o5tc6b1jm

--- n = -1, p = 1000000, loss = 3.672924041748047 ---
mus k.86yj21hv0 vy5iv97dsv  27chhs wfcivme5f08j9k2z4tl4yxgmbtd4po5ta95vst,en82av3v785j8qbttx8x7bscsn

--- n = -1, p = 1200000, loss = 3.7308573722839355 ---
tba,a1ha93e35o'i.bnpfztelsle6a4,sp323swh jw2dtfysq wtlr6b3yxxseuq5psefyj960'5zqzwzqgx y0q8'l1g7fbxg.

--- n = -1, p = 1400000, loss = 3.67

In [8]:
# My implementation of LSTM on character reading - based on the equation above

class BasicLSTM:
    def __init__(self, vocab_size, cell_size, batch_size, continue_training = False, global_step = -1):
        self.batch_size = batch_size
        self.vocab_size = vocab_size
        self.cell_size = cell_size
        self.global_step = global_step
        
        self.global_step = global_step
        self.MODEL_NAME = "./model-2/LSTM"
        self.TEST_SAMPLE_SEQ_LENGTH = 100
        self._construct_networks(vocab_size, cell_size, batch_size, continue_training, global_step)
        
        
        
    def _weight(self, shape, dtype=tf.float32, name=None):
        m = 0
        s = 0.01
        return tf.Variable(tf.random_normal(shape=shape, mean=m, stddev=s, dtype=dtype), dtype, name=name)
    
    def _const(self, shape, name, dtype=tf.float32):
        d0 = shape[0]
        d1 = shape[1]
        tmp = np.zeros(shape=shape)
        tmp[:, 0] = np.ones(shape=[d0, 1])
        return tf.constant(tmp, dtype=dtype, name=name)
    
    def _ohe2char(self, ohe_vec): # takes only the first row in ohe_vec
        assert ohe_vec.shape[1] == self.vocab_size
        chars = 'abcdefghijklmnopqrstuvwxyz .,\'1234567890";'
        choice_id = np.random.choice(self.vocab_size, p=ohe_vec[0,:].ravel())
        return chars[choice_id]
            

    def _construct_networks(self, vocab_size, cell_size, batch_size, continue_training, global_step):
        graph = tf.Graph()
        with graph.as_default():
            x = tf.placeholder(tf.float32, [batch_size, vocab_size], name="x")
            y = tf.placeholder(tf.float32, [batch_size, vocab_size], name="y")
            init_C = tf.placeholder(tf.float32, [batch_size, cell_size], name="init_C")
            init_h = tf.placeholder(tf.float32, [batch_size, vocab_size], name="init_h")
        
            if not continue_training:
                # Fotget gate
                Wf = self._weight([2 * vocab_size, cell_size], name="Wf")
                bf = self._const([1, cell_size], name="bf")
                f = tf.nn.softmax(tf.matmul(tf.concat([init_h, x], axis=1), Wf) + bf, dim=1)

                # Info gate
                Wi = self._weight([2 * vocab_size, cell_size], name="Wi")
                bi = self._const([1, cell_size], name="bi")
                i = tf.nn.softmax(tf.matmul(tf.concat([init_h, x], axis=1), Wi) + bi, dim=1)

                # Next cell state
                Wd = self._weight([2 * vocab_size, cell_size], name="Wd")
                bd = self._const([1, cell_size], name="bd")
                D = tf.tanh(tf.matmul(tf.concat([init_h, x], axis=1), Wd) + bd)

                # Update cell state
                C = tf.add(f * init_C, i * D, name="C")

                # Output layers
                Wo = self._weight([2 * vocab_size, vocab_size], name="Wo")
                bo = self._const([1, vocab_size], name="bo")
                o = tf.nn.softmax(tf.matmul(tf.concat([init_h, x], axis=1), Wo) + bo, dim=1)
                h = tf.multiply(o, tf.tanh(C), name="h")
                hs = tf.nn.softmax(h)

                # Loss function, etc.
                #hs = tf.nn.softmax(h, dim=1) # Convert h into softmax form
                loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=h, name="loss"))
                optimizer = tf.train.AdamOptimizer()
                grad_limit = tf.constant(5.0, dtype=tf.float32, name="grad_limit")
                grads_and_vars = optimizer.compute_gradients(loss)
                clipped_grads_and_vars = []
                for grad, var in grads_and_vars:

                    clipped_grad = tf.clip_by_value(grad, -grad_limit, grad_limit)
                    clipped_grads_and_vars.append((clipped_grad, var))
                train_step = optimizer.apply_gradients(clipped_grads_and_vars, name="train")


                # Session, Saver, etc.
                saver = tf.train.Saver()
                sess = tf.Session()
                sess.run(tf.global_variables_initializer())
                saver.save(sess, self.MODEL_NAME,global_step=0)

            else:
                sess = tf.Session()

                saver = tf.train.import_meta_graph(self.MODEL_NAME + "-{}.meta".format(global_step))
                saver.restore(sess,tf.train.latest_checkpoint('./'))

                graph = tf.get_default_graph()

                x = graph.get_tensor_by_name("x:0")
                y = graph.get_tensor_by_name("y:0")
                init_C = graph.get_tensor_by_name("init_C:0")
                init_h = graph.get_tensor_by_name("init_h:0")
                C = graph.get_tensor_by_name("C:0")
                h = graph.get_tensor_by_name("h:0")
                Wf = graph.get_tensor_by_name("Wf:0")
                Wi = graph.get_tensor_by_name("Wi:0")
                Wd = graph.get_tensor_by_name("Wd:0")
                Wo = graph.get_tensor_by_name("Wo:0")
                J = graph.get_tensor_by_name("J:0")

                train_step = graph.get_tensor_by_name("train:0")
            
        
        # After creation, save to class variables
        self.x = x
        self.y = y
        self.init_C = init_C
        self.init_h = init_h
        self.C = C
        self.h = h
        self.hs = hs
        self.loss = loss
        self.train_step = train_step
        self.saver = saver
        self.sess = sess
        
        

        
    def train(self, steps, training_data, sample = True, sample_every = 200000, save_per_step = 1000):
        save_per_steps = 10
        batch_size = self.batch_size
        vocab_size = self.vocab_size
        cell_size = self.cell_size
        
        for stp in range(steps):
            
            prev_C = np.random.rand(batch_size, cell_size)
            prev_h = np.random.rand(batch_size, vocab_size)
            p = 0
            while p < (len(training_data) - batch_size - 1):
                self.global_step
                fdata = {self.init_C: prev_C, 
                         self.init_h: prev_h,
                         self.x: training_data[p : p + batch_size], 
                         self.y: training_data[p+1 : p+1+batch_size]
                         }
                _, prev_C, prev_h, loss = self.sess.run([self.train_step, self.C, self.h, self.loss], feed_dict = fdata)
                
                p += batch_size
                
            
                if sample and p % sample_every == 0 and stp % 100 == 0:
                    # Perform a trial of sample run 
                    words_outputs = ""
                    for i in range(self.TEST_SAMPLE_SEQ_LENGTH):
                        fdata = {self.init_C: prev_C,
                                self.init_h: prev_h,
                                self.x: training_data[p : p + batch_size],
                                 self.y: training_data[p+1 : p+1+batch_size]
                                }
                        _, vec_ohe, loss = self.sess.run([self.C, self.hs, self.loss], feed_dict = fdata)

                        words_outputs += self._ohe2char(vec_ohe)
                    
                    print ("--- n = {}, p = {}, loss = {} ---".format(self.global_step, p, loss))
                    print ("{}\n".format(words_outputs))
                    
                    
            if self.global_step % save_per_steps == 0:
                self.saver.save(self.sess, self.MODEL_NAME, global_step = self.global_step)
        
            self.global_step += 1
    
    
if __name__ == "__main__":
    print ("Started!")
    lstm = BasicLSTM(vocab_size = 40,
                    cell_size = 40, # They have to be equal. GGWP
                    batch_size = 1000,
                    continue_training = False,
                    global_step = -1)
    training_words = vocab2vec(40, 10 ** 6)
    lstm.train(steps = 10**5, training_data = training_words, sample = True, sample_every = 500000, save_per_step = 100000)    

Started!
shape is: (992170, 40)
--- n = -1, p = 500000, loss = 3.686274766921997 ---
d8w2nsmarlflpt9adog9oye7dde9v3xw4oyb'64,'z9k1u7sic71fd2g8h88bd g507eq4ti,,y7h fh6rg6ou7s0k'fv5gkb7xj

--- n = 99, p = 500000, loss = 3.506497859954834 ---
sgeci5zhhiqf6jql5ksbcvx1imn,d.k nucam80qzin.xxvll8.,oflt 7wsxa87vs7olwn64.pq3e.wuqtw5j0pog6,46dgjvrt

--- n = 199, p = 500000, loss = 3.506502628326416 ---
d' nde3vne2' 'n3oux1 oflslfmwd83bdggbzqhrqc,boxm3vzh niz2yzq gtvbotosdevzrxdfzx,,m1o'm b2ypenrnxrgdn

--- n = 299, p = 500000, loss = 3.506446599960327 ---
o4yhd27.an.annqzk r31fgdfy0j2.'c8np4,6qg7w r3k jst,wbhq6c7y c3emdwtn38i6r0h3jrwwnaqp4j9mo'h squ'iqrc

--- n = 399, p = 500000, loss = 3.5065054893493652 ---
m3pm90da,,gnn'ggo9irnzp.f5y6o lpgm4bnzva48goo04ak2'wyl0meg17wbm3qshxtk7t0'leqlob2nt5lywp0cmf5r''hurb

--- n = 499, p = 500000, loss = 3.5065455436706543 ---
67qzbfi3sfc.8o1xr4s6gvm3m13b99689c56jnp0lns34f.x7m3bq5y0lseib6y nza1f,qu9ak'qeumj d35dsxqruoql,dvffk

--- n = 599, p = 500000, loss = 

KeyboardInterrupt: 

In [ ]:
# Normalize loss through longer batch: 10000
if __name__ == "__main__":
    print ("Started!")
    lstm = BasicLSTM(vocab_size = 40,
                    cell_size = 40, # They have to be equal. GGWP
                    batch_size = 10000,
                    continue_training = False,
                    global_step = -1)
    training_words = vocab2vec(40, 10 ** 6)
    lstm.train(steps = 10**5, training_data = training_words, sample = True, sample_every = 500000, save_per_step = 1000000)    

Started!
shape is: (992170, 40)
--- n = -1, p = 500000, loss = 3.6886608600616455 ---
t'qzrzs8jzqsj2owuwskbz 7b9cekyfehjcv53.xocpa'fk.k.ed3bl3luilix9vt5r1y1uy1u414x4msxchtdj9vj70up'ddf8j

--- n = 99, p = 500000, loss = 3.4971048831939697 ---
c9  ts.8n8c.7g5ti1qnps1nb4nk,04h 31.db'gg.ayzhpn8mng32amdt424 by ,0,965,,n8.zzn8 8nlr,,78m8b.uwnwgns

--- n = 199, p = 500000, loss = 3.497457981109619 ---
0eqto 05ds4s07b0r47vjp6yd35cn25m7fy3spb92s7d9w1dv.kzhxtl4jno5e.1ru7wvj i8t7lnx4rn',65z'9nf5wpksch0ha

--- n = 299, p = 500000, loss = 3.4977023601531982 ---
fv4,vbct0jv,hv74kbx5.oglrggql c3jn'eia,6dnugh4s.ab 0t0si23lwo5 g pg s0uxnbsgbvdcnm30asylxp75t5vr2v1'

--- n = 399, p = 500000, loss = 3.497817039489746 ---
vr ut ieanpofy11qmj,qkwn6i9jgahkym67'i1214jr1ac7d.3nxnw7iycy94zx2xiq1uklny.thxnvmwecvi4wtpw,sw  yoqk

--- n = 499, p = 500000, loss = 3.4978549480438232 ---
ddcpolr497j.u60s6aq6uh94an'sbf6,gxbabz nnb1oeenw542ch2f,83pjkfai7qk,am'wimp5n xg'19w1l11983akn7bj7lv

--- n = 599, p = 500000, loss 